In [1]:
import requests
from bs4 import BeautifulSoup
from scrapfly import ScrapflyClient, ScrapeConfig
import re
import json

api_key = 'scp-live-01fc8534987042f4a2fe553c5cf6a2df'
scrapfly = ScrapflyClient(key=api_key, max_concurrency=2)
url = 'https://www.samsclub.com/p/P03016455'
result = scrapfly.scrape(ScrapeConfig(
    url=url,
    render_js=True,
    country="US",
    asp=True,
    retry=False,
    timeout=30000,
    rendering_wait=5000
))

html_content = result.scrape_result['content']


In [2]:
def extract_json_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tags = soup.find_all('script')

    json_data_list = []

    for script in script_tags:
        script_content = script.string
        if script_content:
            # Use a regular expression to find JSON-like content
            json_pattern = re.compile(r'({.*})', re.DOTALL)
            matches = json_pattern.findall(script_content)
            for match in matches:
                try:
                    json_data = json.loads(match)
                    json_data_list.append(json_data)
                except json.JSONDecodeError:
                    pass  # Ignore invalid JSON

    return json_data_list

json_data_list = extract_json_from_html(html_content)


In [4]:
file_path = r'C:\Users\SudheerRChinthala\circana\Srikanth\Data\SamsClubJSONData.json'
with open(file_path, 'w') as json_file:
    for json_data in json_data_list:
        json.dump(json_data, json_file, indent=4)